

![](https://images.aicrowd.com/raw_images/challenges/banner_file/903/c5e145e1adaa2493032a.jpg)


# 0. Installing aircrowd-cli and downloading Dataset


In [ ]:
!pip install aicrowd-cli
%load_ext aicrowd.magic

In [ ]:
%aicrowd login

In [ ]:
!rm -rf data
!mkdir data
%aicrowd ds dl -c lidar-car-detection -o data

# 1. Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import random

%matplotlib notebook
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

# 2. Reading the Dataset

In [ ]:

train_data = np.load("data/train.npz", allow_pickle=True)
test_data = np.load("data/test.npz", allow_pickle=True)
train_data = train_data['train']
test_data = test_data['test']

train_data.shape, test_data.shape

((400, 2), (601,))

### Visualizing the Dataset

In this section, we will be visualizing a sample 3D lidar data

In [ ]:
# Getting a random 3D lidar sample data
INDEX = random.randint(0, train_data.shape[0] - 1)

# Getting the individual x, y and z points.
x = train_data[INDEX][0][:, 0].tolist()
y = train_data[INDEX][0][:, 1].tolist()
z = train_data[INDEX][0][:, 2].tolist()

# Label for the corrosponding sample ( no. of cars )
label  = train_data[INDEX][1]

# Generating the 3D graph
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(
                                   size=1,       
                                   colorscale='Viridis',
                                   opacity=0.8))])
print("No. of cars : ", label)
fig.show()

No. of cars :  4


## Splitting the dataset

In [ ]:
# Getting the 3d points and flattening the points into 1d array
X = train_data[:, 0]
X = [i.flatten() for i in X]

# labels
y = train_data[:, 1]

In [ ]:
# Splitting the dataset into training and testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Training the model

In [ ]:
model = RandomForestRegressor(verbose=True, n_jobs=-1, n_estimators=70)
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  8.0min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=70, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=True, warm_start=False)

# Validation 

In [ ]:
model.score(X_val, y_val)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  70 out of  70 | elapsed:    0.0s finished


0.2422046810389168

# Generating the predictions

In [ ]:
# Loading the test data

test_data = np.load("/content/data/test.npz", allow_pickle=True)
test_data = test_data['test']

test_data.shape

(601,)

In [ ]:
# flattening the points into 1d array
X_test = X = [i.flatten()    for i in test_data]

In [ ]:
# Generating the predictions
predictions = model.predict(X_test)
predictions.shape

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:    0.1s finished


(601,)

In [ ]:
submission = pd.DataFrame({"label":predictions})
submission

,label
0,2.720000
1,2.803333
2,2.943333
3,3.270000
4,2.073333
...,...
596,2.466667
597,1.863333
598,2.130000
599,2.610000


In [ ]:
# Saving the predictions
!rm -rf assets
!mkdir assets
submission.to_csv(os.path.join("assets", "submission.csv"))

# Submitting our Predictions
**Note : Please save the notebook before submitting it (Ctrl + S)**

In [ ]:
%aicrowd notebook submit -c lidar-car-detection -a assets --no-verify

/usr/local/lib/python3.7/dist-packages/aicrowd/notebook/helpers.py:361: UserWarning: `%aicrowd` magic command can be used to save the notebook inside jupyter notebook/jupyterLab environment and also to get the notebook directly from the frontend without mounting the drive in colab environment. You can use magic command to skip mounting the drive and submit using the code below:
 %load_ext aicrowd.magic
%aicrowd notebook submit -c lidar-car-detection -a assets --no-verify
  warnings.warn(description + code)
Using notebook: Lidar Car Prediction for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...
submission.zip ━━━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 1.2/1.2 MB • 1.2 MB/s • 0:00:00
                                                   ╭─────────────────────────╮                                                    
                                                   │ Successfully submitted! │                                                  